In [1]:
using Pkg

Pkg.activate(".")

  Activating project at `~/Documents/Uni/Master/MA/visual_analysis`


In [2]:
using GLMakie
using GeoMakie
using EmpiricalOrthogonalFunctions
using NCDatasets 
using Dates
using BenchmarkTools
using Statistics

In [36]:
include("utils.jl")

get_sliding_time_scopes_by_threshold (generic function with 2 methods)

In [32]:
# base_path = "/home/denis/workspace/data/ivt_fields_v1"
base_path = "/mnt/bigdrive/Datasets/ivt_fields_v1"
# base_path = "/home/denis/workspace/datasets/ps_data_fast"

"/mnt/bigdrive/Datasets/ivt_fields_v1"

In [37]:
winter_months = [12, 1, 2, 3]

tl_data = filter_by_date(build_timeline_data(base_path, 20, "ssp126", "ssp585"; file_range_selection = :)) do time_element

    for wm in winter_months
        if month(time_element) == wm
            return true
        end
    end
    return false
end

TimelineData(Union{Missing, AbstractFloat}[-90.0, -88.125, -86.25, -84.375, -82.5, -80.625, -78.75, -76.875, -75.0, -73.125  …  22.5, 24.375, 26.25, 28.125, 30.0, 31.875, 33.75, 35.625, 37.5, 39.375], Union{Missing, AbstractFloat}[21.450475037398185, 23.31573072614093, 25.180985581270594, 27.04623949994481, 28.91149236871774, 30.77674406172325, 32.64199443851768, 34.50724334150103, 36.37249059281224, 38.23773599056483  …  62.48557052203639, 64.35073040887207, 66.2158721139987, 68.08099098565125, 69.94608064698343, 71.81113211427447, 73.67613231320912, 75.54106145287895, 77.4058880820788, 79.27055903485967], Union{Missing, DateTime}[DateTime("2015-01-01T06:00:00"), DateTime("2015-01-01T12:00:00"), DateTime("2015-01-01T18:00:00"), DateTime("2015-01-02T00:00:00"), DateTime("2015-01-02T06:00:00"), DateTime("2015-01-02T12:00:00"), DateTime("2015-01-02T18:00:00"), DateTime("2015-01-03T00:00:00"), DateTime("2015-01-03T06:00:00"), DateTime("2015-01-03T12:00:00")  …  DateTime("2100-12-29T18:00:

In [11]:
winter_months = [12, 1, 2, 3]

tl_data = filter_by_date(build_timeline_data(base_path, 10, "ssp585"; file_range_selection = :, data_field_id = "ps")) do time_element

    for wm in winter_months
        if month(time_element) == wm
            return true
        end
    end
    return false
end

TimelineData(Union{Missing, AbstractFloat}[-90.0, -88.125, -86.25, -84.375, -82.5, -80.625, -78.75, -76.875, -75.0, -73.125  …  22.5, 24.375, 26.25, 28.125, 30.0, 31.875, 33.75, 35.625, 37.5, 39.375], Union{Missing, AbstractFloat}[21.450475037398185, 23.31573072614093, 25.180985581270594, 27.04623949994481, 28.91149236871774, 30.77674406172325, 32.64199443851768, 34.50724334150103, 36.37249059281224, 38.23773599056483  …  62.48557052203639, 64.35073040887207, 66.2158721139987, 68.08099098565125, 69.94608064698343, 71.81113211427447, 73.67613231320912, 75.54106145287895, 77.4058880820788, 79.27055903485967], Union{Missing, DateTime}[DateTime("2015-01-01T06:00:00"), DateTime("2015-01-01T12:00:00"), DateTime("2015-01-01T18:00:00"), DateTime("2015-01-02T00:00:00"), DateTime("2015-01-02T06:00:00"), DateTime("2015-01-02T12:00:00"), DateTime("2015-01-02T18:00:00"), DateTime("2015-01-03T00:00:00"), DateTime("2015-01-03T06:00:00"), DateTime("2015-01-03T12:00:00")  …  DateTime("2100-12-29T18:00:

In [23]:
# animate_timeline(tline_data, "scenario_comparison_2.mp4"; colormap = :managua100, resolution = (1000, 1000))
animate_timeline(tl_data, "scenario_comparison_end.mp4"; colormap = :managua100)

In [30]:
include("visual_utils.jl")

show_eof_modes_of_timeline (generic function with 1 method)

In [ ]:

last_spatial_data = nothing

for scope in get_sliding_time_scopes_by_threshold(tl_data.time, 4)
    spatial_modes, temporal_modes, variability_percentage = get_eof_of_datachunk(tl_data.datasets[1].data[:, :, scope]; nmodes = 4)
    if !isnothing(last_spatial_data)
        diff = abs.(last_spatial_data .- spatial_modes)
        println("Mean diff: $(mean(diff))")
        println("Max diff: $(maximum(diff))")
        println("Min diff: $(minimum(diff))")
    end
    last_spatial_data = spatial_modes
end

In [46]:
scopes = get_sliding_time_scopes_by_threshold(tl_data.time, 4)

figs = []
for i in [1, length(scopes)]
    spatial_modes, temporal_modes, variability_percentage = get_eof_of_datachunk(tl_data.datasets[2].data[:, :, scopes[i]]; nmodes = 4)
    push!(figs, build_figure(spatial_modes[:, :, 1], (-90, 40), (20, 80); title = "$i ssp585"))
end

In [10]:
function normalize_tl_data(tl_data)

    normalized_ds = ScenarioData[]
    for ds in tl_data.datasets
        min, max = extrema(ds.data)
        res = (ds.data .- min) ./ (max .- min)
        push!(normalized_ds, ScenarioData(ds.name, res))
    end

    return TimelineData(tl_data.lons, tl_data.lats, tl_data.time, normalized_ds)
end

normalize_tl_data (generic function with 1 method)

In [21]:
function centralize_tl_data(tl_data; dims = 3)

    normalized_ds = ScenarioData[]
    for ds in tl_data.datasets
        res = ds.data .- mean(ds.data, dims = dims)
        push!(normalized_ds, ScenarioData(ds.name, res))
    end

    return TimelineData(tl_data.lons, tl_data.lats, tl_data.time, normalized_ds)
end

centralize_tl_data (generic function with 1 method)

In [11]:
normalized_tl_data = normalize_tl_data(tl_data)

TimelineData(Union{Missing, AbstractFloat}[-90.0, -88.125, -86.25, -84.375, -82.5, -80.625, -78.75, -76.875, -75.0, -73.125  …  22.5, 24.375, 26.25, 28.125, 30.0, 31.875, 33.75, 35.625, 37.5, 39.375], Union{Missing, AbstractFloat}[21.450475037398185, 23.31573072614093, 25.180985581270594, 27.04623949994481, 28.91149236871774, 30.77674406172325, 32.64199443851768, 34.50724334150103, 36.37249059281224, 38.23773599056483  …  62.48557052203639, 64.35073040887207, 66.2158721139987, 68.08099098565125, 69.94608064698343, 71.81113211427447, 73.67613231320912, 75.54106145287895, 77.4058880820788, 79.27055903485967], Union{Missing, DateTime}[DateTime("2015-01-01T06:00:00"), DateTime("2015-01-01T12:00:00"), DateTime("2015-01-01T18:00:00"), DateTime("2015-01-02T00:00:00"), DateTime("2015-01-02T06:00:00"), DateTime("2015-01-02T12:00:00"), DateTime("2015-01-02T18:00:00"), DateTime("2015-01-03T00:00:00"), DateTime("2015-01-03T06:00:00"), DateTime("2015-01-03T12:00:00")  …  DateTime("2100-12-29T18:00:

In [22]:
time_centered_tldata = centralize_tl_data(tl_data)

TimelineData(Union{Missing, AbstractFloat}[-90.0, -88.125, -86.25, -84.375, -82.5, -80.625, -78.75, -76.875, -75.0, -73.125  …  22.5, 24.375, 26.25, 28.125, 30.0, 31.875, 33.75, 35.625, 37.5, 39.375], Union{Missing, AbstractFloat}[21.450475037398185, 23.31573072614093, 25.180985581270594, 27.04623949994481, 28.91149236871774, 30.77674406172325, 32.64199443851768, 34.50724334150103, 36.37249059281224, 38.23773599056483  …  62.48557052203639, 64.35073040887207, 66.2158721139987, 68.08099098565125, 69.94608064698343, 71.81113211427447, 73.67613231320912, 75.54106145287895, 77.4058880820788, 79.27055903485967], Union{Missing, DateTime}[DateTime("2015-01-01T06:00:00"), DateTime("2015-01-01T12:00:00"), DateTime("2015-01-01T18:00:00"), DateTime("2015-01-02T00:00:00"), DateTime("2015-01-02T06:00:00"), DateTime("2015-01-02T12:00:00"), DateTime("2015-01-02T18:00:00"), DateTime("2015-01-03T00:00:00"), DateTime("2015-01-03T06:00:00"), DateTime("2015-01-03T12:00:00")  …  DateTime("2100-12-29T18:00:

In [38]:
show_eof_modes_of_timeline(tl_data, 4, 4, "eof_modes.mp4"; colormap = :managua100, framerate = 2, resolution = (2000, 1000), eof_center = true)